In [ ]:
# Import Dependencies
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [ ]:
# Create Engine for census data
engine = create_engine("sqlite:///../../data/FPA_FOD_20170508.sqlite")
conn = engine.connect()

In [ ]:
# Query All Records in the the Fires Table
fires_data = pd.read_sql("SELECT * FROM Fires", conn)

In [ ]:
len(fires_data)

In [ ]:
fires_data.head(10)

In [ ]:
biggest_fires = fires_data.sort_values(by = 'FIRE_SIZE',ascending = False)
biggest_fires_trimmed = biggest_fires[["FIRE_NAME","FIRE_YEAR","DISCOVERY_DATE","STATE",
                                    "STAT_CAUSE_DESCR","FIRE_SIZE","FIRE_SIZE_CLASS",
                                    "LATITUDE","LONGITUDE","FIPS_NAME"]]

biggest_fires_trimmed.head(10)

In [ ]:
top_20_fires = biggest_fires_trimmed[:20]
epoch = pd.to_datetime(0, unit='s').to_julian_date()
top_20_fires["DISCOVERY_DATE"] = pd.to_datetime(top_20_fires["DISCOVERY_DATE"] - epoch, unit='D')
top_20_fires_df = top_20_fires.reset_index()
top_20_fires_df

In [ ]:
output_path_top_20_fires = "output/top_20_fires.csv"
top_20_fires_df.to_csv(output_path_top_20_fires,index=False)

In [ ]:
fires_data_trimmed = fires_data[["OBJECTID","FIRE_YEAR","DISCOVERY_DATE","STATE"
                                    ,"FIRE_SIZE","FIRE_SIZE_CLASS"]]
fires_data_trimmed.head(10)

In [ ]:
epoch = pd.to_datetime(0, unit='s').to_julian_date()
fires_data_trimmed["DISCOVERY_DATE"] = pd.to_datetime(fires_data_trimmed["DISCOVERY_DATE"] - epoch, unit='D')
fires_data_trimmed_converted = fires_data_trimmed

In [ ]:
fires_data_yearly_df = pd.DataFrame(fires_data_trimmed_converted)
print(len(fires_data_yearly_df))
fires_data_yearly_df.head(5) 


In [ ]:
# grouping by year
year_group = fires_data_yearly_df.groupby(['FIRE_YEAR'])
year_group

In [ ]:
# grouping by year & state
year_state_group = fires_data_yearly_df.groupby(['FIRE_YEAR','STATE'])
year_state_group

In [ ]:
# total fires by year
total_fires_year = year_group['OBJECTID'].count()
total_fires_year_df = pd.DataFrame(total_fires_year).rename(columns={"OBJECTID": "COUNT_FIRES"})
total_fires_year_df

In [ ]:
# total fires by year by state
total_fires_year_state = year_state_group['OBJECTID'].count()
total_fires_year_state_df = pd.DataFrame(total_fires_year_state).rename(columns={"OBJECTID": "COUNT_FIRES"})
total_fires_year_state_df

In [ ]:
# total fire size by year
total_fire_size_year = year_group['FIRE_SIZE'].sum()
total_fire_size_year_df = pd.DataFrame(total_fire_size_year)
total_fire_size_year_df

In [ ]:
# total fire size by year by state
total_fire_size_year_state = year_state_group['FIRE_SIZE'].sum()
total_fire_size_year_state_df = pd.DataFrame(total_fire_size_year_state)
total_fire_size_year_state_df

In [ ]:
# merging count of fires and sum of size dataframes
fire_size_count_year_state = total_fire_size_year_state_df.merge(total_fires_year_state_df
                                    ,left_index=True,right_index=True)
fire_size_count_year_state_df = pd.DataFrame(fire_size_count_year_state)
fire_size_count_year_state_df

In [ ]:
fire_size_count_year_state_df['AVG_FIRE_SIZE'] = fire_size_count_year_state_df['FIRE_SIZE'] / fire_size_count_year_state_df['COUNT_FIRES']
fire_size_count_year_state_df

In [ ]:
year_df_1 = fire_size_count_year_state_df.reset_index()
year_df_2 = fire_size_count_year_state_df.groupby(["FIRE_YEAR"]).sum()
year_df_2["STATE"] = "ALL STATES"

year_df_3 = year_df_2.reset_index()

first_column = year_df_3.pop('STATE')
year_df_3.insert(0, 'STATE', first_column)
# year_df_4 = year_df_3.columns("FIRE_YEAR","STATE","FIRE_SIZE","COUNT_FIRES","AVG_FIRE_SIZE")

year_df_3

In [ ]:
first_column = year_df_3.pop('FIRE_YEAR')
year_df_3.insert(0, 'FIRE_YEAR', first_column)

year_df_3

In [ ]:
year_df_3["RUNNING_COUNT_SIZE"] = year_df_3["FIRE_SIZE"].cumsum(axis = 0) 
year_df_3["RUNNING_COUNT_FIRES"] = year_df_3["COUNT_FIRES"].cumsum(axis = 0) 
year_df_3["PRIOR_RUNNING_SIZE"] = year_df_3["RUNNING_COUNT_SIZE"] - year_df_3["FIRE_SIZE"]
year_df_3["PRIOR_RUNNING_COUNT"] = year_df_3["RUNNING_COUNT_FIRES"] - year_df_3["COUNT_FIRES"]
year_running_df = year_df_3
year_running_df

In [ ]:
fire_size_count_year_all_state_df = year_df_1.append(year_df_3)
fire_size_count_year_all_state_df

In [ ]:
# exporting total fires by year by state
output_path_fires_year_state = "output/total_fires_yearly_by_state.csv"
fire_size_count_year_all_state_df.to_csv(output_path_fires_year_state,index=False)